In [ ]:
#!/usr/bin/env python3
# coding=utf-8
"""
Live Fruit Detection + DOFBOT Sorting
------------------------------------
Runs YOLOv8 on a live camera feed and lets the user press "s" to
sort the last detected fruit using the DOFBOT arm.
Supported classes: apple, banana, orange, lemon, pomegranate, tomato.
"""


In [ ]:
import time
import cv2
from ultralytics import YOLO
from Arm_Lib import Arm_Device

print('Initialising DOFBOT arm...')
Arm = Arm_Device()
time.sleep(0.3)
print('DOFBOT initialised.')


In [ ]:
def arm_clamp_block(enable: int) -> None:
    """Open (0) or close (1) the DOFBOT gripper."""
    if enable == 0:
        Arm.Arm_serial_servo_write(6, 60, 400)
    else:
        Arm.Arm_serial_servo_write(6, 135, 400)
    time.sleep(0.4)

def arm_move(poses, t: int = 700) -> None:
    """Move servos 1–5 to the angles defined in poses."""
    for i in range(5):
        servo_id = i + 1
        angle = poses[i]
        move_time = int(t * 1.2) if servo_id == 5 else t
        Arm.Arm_serial_servo_write(servo_id, angle, move_time)
        time.sleep(0.01)
    time.sleep(t / 1000.0)

def arm_move_up() -> None:
    """Lift the arm to a neutral 'up' position."""
    Arm.Arm_serial_servo_write(2, 90, 1500)
    Arm.Arm_serial_servo_write(3, 90, 1500)
    Arm.Arm_serial_servo_write(4, 90, 1500)
    time.sleep(0.3)

print('Arm helper functions loaded.')


In [ ]:
p_home = [90, 130, 0, 0, 90]
p_pick_top = [90, 80, 50, 50, 270]
p_pick = [90, 53, 33, 36, 270]

p_basket_yellow = [65, 22, 64, 56, 270]
p_basket_red    = [117, 19, 66, 56, 270]
p_basket_green  = [136, 66, 20, 29, 270]
p_basket_blue   = [44, 66, 20, 28, 270]

FRUIT_TO_BASKET = {
    'apple':        p_basket_green,
    'banana':       p_basket_yellow,
    'orange':       p_basket_red,
    'lemon':        p_basket_blue,
    'pomegranate':  p_basket_red,
    'tomato':       p_basket_green,
}

print('Arm poses and fruit mapping configured.')


In [ ]:
def grab_from_pick_and_place(target_pose, label: str = '') -> None:
    """Execute a full pick-and-place cycle for one fruit."""
    if target_pose is None:
        print('No valid basket pose. Aborting.')
        return

    print(f'Sorting fruit: {label}')

    arm_clamp_block(0)
    arm_move(p_home, 1000)

    arm_move(p_pick_top, 1000)
    arm_move(p_pick, 1000)

    arm_clamp_block(1)
    arm_move(p_pick_top, 1000)

    arm_move(target_pose, 1100)

    arm_clamp_block(0)

    arm_move_up()
    arm_move(p_home, 1100)

    print('Sorting cycle completed.')


In [ ]:
print('Loading YOLO model (best.pt)...')
model = YOLO('best.pt')
print('YOLO model loaded.')


In [ ]:
def try_camera(idx: int):
    cap = cv2.VideoCapture(idx)
    time.sleep(1)
    if cap.isOpened():
        ok, _ = cap.read()
        if ok:
            return cap
    return None

cap = None
for cam_idx in [0, 1, 2]:
    print(f'Trying camera index {cam_idx}...')
    cap = try_camera(cam_idx)
    if cap:
        print(f'Camera opened on index {cam_idx}.')
        break

if not cap:
    raise RuntimeError('No camera could be opened.')

cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
print('Camera initialised.')


In [ ]:
VALID_CLASSES = list(FRUIT_TO_BASKET.keys())

def detect_fruit(frame):
    """Return (class_name, confidence) for the best valid detection."""
    results = model(frame, imgsz=640, conf=0.5, verbose=False)
    best_cls = None
    best_conf = 0.0
    for r in results:
        for box in r.boxes:
            cls_id = int(box.cls[0])
            conf = float(box.conf[0])
            name = r.names[cls_id].lower()
            if name in VALID_CLASSES and conf > best_conf:
                best_cls = name
                best_conf = conf
    return best_cls, best_conf

print('Detection function ready.')


In [ ]:
print('Starting live detection... Press s to sort, q to quit.')

last_detected = None

try:
    while True:
        ok, frame = cap.read()
        if not ok:
            continue

        cls, conf = detect_fruit(frame)
        if cls:
            last_detected = cls
            text = f"{cls} ({conf:.2f})"
            cv2.putText(frame, text, (20, 40),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 0), 2)

        cv2.imshow('Live Fruit Detection', frame)
        key = cv2.waitKey(1) & 0xFF

        if key == ord('q'):
            print('Exiting...')
            break

        if key == ord('s'):
            if last_detected:
                pose = FRUIT_TO_BASKET.get(last_detected)
                grab_from_pick_and_place(pose, label=last_detected)
            else:
                print('No fruit detected to sort yet.')

except KeyboardInterrupt:
    print('Interrupted by user.')

finally:
    cap.release()
    cv2.destroyAllWindows()
    try:
        del Arm
    except Exception:
        pass
    print('System shut down.')
